# D05 Đối xử với dữ liệu NA

## Mục đích

Giới thiệu cách phát hiện và các biện pháp đối xử với dữ liệu NA. Xem thêm ở [đây](https://pandas.pydata.org/docs/user_guide/missing_data.html).


## Phát hiện dữ liệu NA

Để kiểm tra xem một giá trị có phải là NA hay không, bạn có thể dùng hàm `pandas.isna()`. Ngược lại, nếu muốn kiểm tra một giá trị KHÔNG phải là NA hay không, chúng ta dùng hàm `pandas.notna()`.

In [1]:
import pandas as pd

a, b = 1, None

print(pd.isna(a), pd.notna(b))

False False


Các hàm này cũng sử dụng được cho series và data frame, nhưng đã được tích hợp vào trong các lớp dữ liệu này.

In [2]:
d = pd.read_excel("../assets/hrm.xlsx", index_col="id")
d["eso_LA"].isna()

id
223     False
236     False
256     False
296     False
310     False
        ...  
4200     True
4214    False
4216    False
4220    False
4240    False
Name: eso_LA, Length: 330, dtype: bool

### Thống kê số liệu NA

Bạn có thể thống kê xem mỗi cột có bao nhiêu bản ghi chứa giá trị không phải là NA (non-null) bằng hàm `info()`. Trong ví dụ dưới đây, mình chỉ hiển thị 5 cột đầu tiên.

In [3]:
d.iloc[:, :5].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 330 entries, 223 to 4240
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   sex        330 non-null    int64         
 1   yob        330 non-null    int64         
 2   height     316 non-null    float64       
 3   weight     324 non-null    float64       
 4   date_exam  330 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(2)
memory usage: 15.5 KB


Nhưng mình ưa thích cách sau đây hơn.

In [4]:
d.iloc[:, :5].isna().sum()

sex           0
yob           0
height       14
weight        6
date_exam     0
dtype: int64

Hàm `sum()` mà chúng ta sử dụng ở trên là một hàm tổng hợp dữ liệu. Mặc định, nó sẽ tính tổng của các giá trị dọc theo các hàng (đối số `axis=0`), có nghĩa là với một data frame có hai chiều thì sẽ tính ra tổng các hàng của mỗi cột. Trong trường hợp muốn tính các giá trị dọc theo các cột, chúng ta thiết lập đối số `axis=1`; ví dụ tính tổng điểm FSSG.

In [5]:
d.filter(like="q_fssg_").sum(axis=1)

id
223     10.0
236     11.0
256     17.0
296     16.0
310     13.0
        ... 
4200    18.0
4214     7.0
4216    11.0
4220     7.0
4240     4.0
Length: 330, dtype: float64

### Kiểm tra NA với nhiều điều kiện

Trong ví dụ về kiểm tra giá trị NA, chúng ta sử dụng hàm `sum()` cho một series kiểu `bool`. Như đã chia sẻ ở những bài trước, các phép toán số học sẽ tự động chuyển dữ liệu kiểu `bool` về `int` (với các giá trị 0 và 1). Đó là lí do vì sao chúng ta có thể đếm được số bản ghi có giá trị.

Lợi dụng sự tiện lợi này, bạn có thể đếm số giá trị NA với những điều kiện phức tạp hơn. Chẳng hạn chúng ta đếm xem có bao nhiêu bản ghi có missing chiều cao hoặc cân nặng.

In [6]:
(d["weight"].isna() | d["height"].isna()).sum()

17

### `any()` và `all()`

Trong dòng lệnh trên, chúng ta kiểm tra NA cho cột `"weight"` và cột `"height"`, sau đó dùng toán tử OR để kiểm tra bản ghi nào có thể missing ít nhất một trong hai, kết quả trả về sẽ là một series kiểu `bool`. Và cuối cùng, chúng ta dùng hàm `sum()` để đếm số giá trị `True` trong series này.

Nếu tất cả các điều kiện của bạn đều chỉ là kiểm tra NA, bạn sẽ thấy việc gõ đi gõ lại `isna()` và toán tử OR là điều hết sức bất tiện. May mắn thay, chúng ta có nhiều cách để rút gọn công việc này.

Đầu tiên, hãy làm quen với hàm `any()` dùng để kiểm tra trong series có bất kì một giá trị `True` nào hay không.

In [7]:
s = pd.Series([1, 1, 1, 0, 0, 1], dtype=bool)
print(s)
s.any()

0     True
1     True
2     True
3    False
4    False
5     True
dtype: bool


True

Và hàm `all()` dùng để kiểm tra tất cả các giá trị có phải là `True` hay không.

In [8]:
s.all()

False

Chúng ta có thể sử dụng tính năng này để kiểm tra xem mỗi bản ghi có bất kì giá trị NA nào trong các cột điểm FSSG hay không. Nhớ rằng chúng ta sẽ kiểm tra các giá trị trong từng **cột** của mỗi hàng, do đó cần thiết lập đối số `axis=1` cho hàm `any()`.

In [9]:
d.filter(like="q_fssg_").isna().any(axis=1).sum()

10

Một cách làm khác là đếm số NA của mỗi hàng, sau đó dùng điều kiện "số NA > 0" để phát hiện các hàng có NA.

In [10]:
d.filter(like="q_fssg_").isna().sum(axis=1).gt(0).sum()

10

Ở trên đây chúng ta dùng `filter()` để chọn các cột có tên thỏa mãn điều kiện nào đó. Bạn vẫn có thể áp dụng các cách ở trên cho data frame được slice. Mình có một thói quen tách danh sách dùng để slice thành một biến riêng để cho gọn và dễ chỉnh sửa mã lệnh.

In [11]:
cols = ["weight", "height"]
print(d[cols].isna().sum(axis=1).gt(0).sum())
print(d[cols].isna().any(axis=1).sum())

17
17


### Slice danh sách chứa giá trị NA

Trong khi làm sạch số liệu, chúng ta sẽ thường phải quay trở lại hồ sơ gốc để kiểm tra các bản ghi có missing. Lọc ra danh sách của các bản ghi này là cần thiết.

In [12]:
cols = ["weight", "height"]
# Bản ghi nào có weight hoặc height missing
na_check = d[cols].isna().any(axis=1)

# Lọc danh sách có ID, giới tính, năm sinh
# và ngày vào nghiên cứu để kiểm tra
d.loc[na_check, ["sex", "yob", "date_exam"]]

,sex,yob,date_exam
id,,,
401,0,1966,2019-04-02
587,0,1963,2019-03-28
920,0,1979,2019-03-02
1077,0,1982,2019-04-18
1504,0,1955,2019-04-12
1773,0,1963,2019-03-26
2058,0,1964,2019-03-09
2201,0,1970,2019-03-30
2289,0,1978,2019-03-26


## Đối xử với dữ liệu NA

### Xóa các bản ghi NA

Trong trường hợp bạn không cần giữ lại các bản ghi có giá trị NA (ví dụ, trong complete case analysis), bạn có thể loại bỏ các hàng trong data frame bằng `dropna()`.

In [13]:
d_dropna = d.dropna()
d_dropna.iloc[:, :5].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102 entries, 256 to 4240
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   sex        102 non-null    int64         
 1   yob        102 non-null    int64         
 2   height     102 non-null    float64       
 3   weight     102 non-null    float64       
 4   date_exam  102 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(2)
memory usage: 4.8 KB


Mặc định, hàm `dropna()` sẽ xóa các hàng (`axis=0`) chứa bất kì giá trị NA ở bất kì nào (`how="any"`) trong tất cả các cột. Trong trường hợp, bạn muốn loại bỏ các hàng chứa NA chỉ ở trong một số cột (ví dụ, các cột outcome), chúng ta thêm đối số `subset`.

In [14]:
d_dropna = d.dropna(subset=["weight", "height"])
d_dropna.iloc[:, :5].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 313 entries, 223 to 4240
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   sex        313 non-null    int64         
 1   yob        313 non-null    int64         
 2   height     313 non-null    float64       
 3   weight     313 non-null    float64       
 4   date_exam  313 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(2)
memory usage: 14.7 KB


### Thay số liệu NA bằng giá trị khác

Trong một số kịch bản khác, bạn lại muốn thay giá trị NA bằng một giá trị nào đó. Ví dụ trong single imputation, bạn muốn lấp đầy các giá trị missing bằng trung bình hoặc trung vị của các giá trị không missing.

Trong ví dụ đơn giản dưới đây, chúng ta sẽ lấp đầy các bản ghi có missing trong các câu hỏi FSSG bằng giá trị 0. Trước hết, hãy lấy một ví dụ bản ghi đầu tiên trong data frame có missing FSSG.

In [15]:
# Lấy danh sách các cột FSSG để slice nhiều lần
fssg_cols = list(d.filter(like="q_fssg_").columns)
na_check = d[fssg_cols].isna().any(axis=1)
d.loc[na_check, fssg_cols].iloc[0]

q_fssg_01_nongrat            2.0
q_fssg_02_dayhoi             2.0
q_fssg_03_nangbung           0.0
q_fssg_04_xoanguc            0.0
q_fssg_05_metsauan           0.0
q_fssg_06_nongratsauan       NaN
q_fssg_07_hong               2.0
q_fssg_08_daylucan           0.0
q_fssg_09_nuotnghen          0.0
q_fssg_10_dichtraolen        0.0
q_fssg_11_onhieu             4.0
q_fssg_12_nongratcuixuong    0.0
Name: 326, dtype: float64

Như vậy bản ghi của người có ID 326 có một giá trị NA ở câu hỏi 6 của điểm FSSG. Bây giờ chúng ta sẽ lấp đầy missing bằng các giá trị 0 bằng hàm `fillna()` và kiểm tra lại bản ghi 326. Trong ví dụ dưới đây, mình lưu số liệu đã lấp đầy vào một data frame khác; bạn có thể lưu vào chính data frame gốc bằng phép gán `d[fssg_cols] = ...`.

In [16]:
d_fssg_fill = d[fssg_cols].fillna(0)
d_fssg_fill.loc[326]

q_fssg_01_nongrat            2.0
q_fssg_02_dayhoi             2.0
q_fssg_03_nangbung           0.0
q_fssg_04_xoanguc            0.0
q_fssg_05_metsauan           0.0
q_fssg_06_nongratsauan       0.0
q_fssg_07_hong               2.0
q_fssg_08_daylucan           0.0
q_fssg_09_nuotnghen          0.0
q_fssg_10_dichtraolen        0.0
q_fssg_11_onhieu             4.0
q_fssg_12_nongratcuixuong    0.0
Name: 326, dtype: float64

Tuy nhiên, nếu bạn kiểm tra kĩ hơn, bạn sẽ phát hiện ra có những người không trả lời bộ câu hỏi FSSG. Trong bộ số liệu, chúng ta có cột `"fssg"` ghi nhận việc này (`"Y"` là có trả lời, `"N"` là không trả lời, và trong trường hợp này, toàn bộ các cột điểm FSSG phải là NA).

In [17]:
d["fssg"].eq("N").sum()

2

Do đó, để lấp đầy một cách chính xác, bạn cần lọc các hàng có `"N"` để không lấp đầy NA của những hàng này. Một cách làm khác là sau khi lấp đầy thì chúng ta dùng `mask` để trả lại giá trị NA cho các dòng đó.

In [18]:
# CÁCH 1: Khi lọc các hàng để lấp đầy (dùng slicing)
# bạn phải nhớ gán lại vào data frame được slice cùng điều kiện
#
# Mình tạo một bản sao của data frame vì mình muốn giữ số liệu gốc
d_copy = d.copy(deep=True)
fssg_yes = d_copy["fssg"].eq("N")
d_copy.loc[fssg_yes, fssg_cols] = d_copy.loc[fssg_yes, fssg_cols].fillna(0)

# CÁCH 2: Dùng mask() tuy tiện hơn nhưng mất thêm một lần thay thế dữ liệu.
# Nếu không khai báo đối số other, mặc định mask() sẽ thay thế bằng giá trị NA.
d_fssg_fill = d[fssg_cols].fillna(0).mask(d["fssg"].eq("N"))
print(d_fssg_fill.isna().any(axis=1).sum())
d_fssg_fill.loc[326]

2


q_fssg_01_nongrat            2.0
q_fssg_02_dayhoi             2.0
q_fssg_03_nangbung           0.0
q_fssg_04_xoanguc            0.0
q_fssg_05_metsauan           0.0
q_fssg_06_nongratsauan       0.0
q_fssg_07_hong               2.0
q_fssg_08_daylucan           0.0
q_fssg_09_nuotnghen          0.0
q_fssg_10_dichtraolen        0.0
q_fssg_11_onhieu             4.0
q_fssg_12_nongratcuixuong    0.0
Name: 326, dtype: float64

Sau khi thay thế NA, các giá trị của chúng ta vẫn là `float` (series chứa giá trị NA luôn có kiểu `float` mặc dù các giá trị số còn lại có thể là `int`). Để tự động chuyển lại thành giá trị `int`, bạn thêm đối số `downcast="infer"`, Pandas sẽ tự động kiểm tra xem series nào có thể convert về `int`. Để mô phỏng ví dụ này, mình sẽ bỏ hàm `mask()` ở cuối.

In [19]:
d_fssg_fill = d[fssg_cols].fillna(0, downcast="infer")
d_fssg_fill.loc[326]

q_fssg_01_nongrat            2
q_fssg_02_dayhoi             2
q_fssg_03_nangbung           0
q_fssg_04_xoanguc            0
q_fssg_05_metsauan           0
q_fssg_06_nongratsauan       0
q_fssg_07_hong               2
q_fssg_08_daylucan           0
q_fssg_09_nuotnghen          0
q_fssg_10_dichtraolen        0
q_fssg_11_onhieu             4
q_fssg_12_nongratcuixuong    0
Name: 326, dtype: int64

## Tính toán với NA

Pandas có hai cách đối xử với giá trị NA khi tính toán: bỏ qua (skip) hoặc trả về NA. Với các hàm tổng hợp dữ liệu, mặc định các giá trị NA sẽ bị bỏ qua. Nếu bạn vẫn muốn giữ các giá trị này và trả về NA, hãy thiết lập đối số `skipna=False`.

In [20]:
d["height"].median()

1.58

In [21]:
d["height"].median(skipna=False)

nan

Một ví dụ khác là trong trường hợp điểm FSSG bị missing, chúng ta sẽ không tính được tổng điểm. Nhưng nếu không thiết lập `skipna=False`, hàm `sum()` vẫn sẽ tự động bỏ qua NA và cộng các giá trị còn lại vào với nhau.

In [22]:
fssg_cols = list(d.filter(like="q_fssg_").columns)
na_check = d[fssg_cols].isna().any(axis=1)

# skipna=True (mặc định)
d[fssg_cols].sum(axis=1).loc[na_check]

id
326     10.0
446      8.0
448     10.0
496     16.0
805     12.0
1182     0.0
1244    12.0
1254    16.0
1308     4.0
2487     0.0
dtype: float64

In [23]:
# skipna=False
d[fssg_cols].sum(axis=1, skipna=False).loc[na_check]

id
326    NaN
446    NaN
448    NaN
496    NaN
805    NaN
1182   NaN
1244   NaN
1254   NaN
1308   NaN
2487   NaN
dtype: float64

---

[Bài trước](./04_replace.ipynb) - [Danh sách bài](../README.md) - [Bài sau](./06_groupby.ipynb)